In [ ]:
import json
import base64
from datetime import datetime, timedelta
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
import webbrowser
import time
import os

# Check if the person is more than 17 years old and has no ludopathy
def check_eligibility(person):
    try:
        birth_date = datetime.strptime(person['birthdate'], '%Y-%m-%d')
        age = (datetime.now() - birth_date).days / 365
        has_ludopathy = person['ludopathy'].lower() == 'yes'
        if age < 18:
            return "child-friendly" 
        else:
            if has_ludopathy:
                return "adult_with_ludopathy"
            else:
                return "adult_without_ludopathy"
    except ValueError:
        return "invalid_data"

# Encrypt user attributes using the authority's public key
def encrypt_attributes(attributes, public_key):
    encrypted_attributes = []
    for attribute in attributes:
        encrypted = public_key.encrypt(
            attribute.encode('utf-8'),
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        encrypted_attributes.append(base64.b64encode(encrypted).decode('utf-8'))
    return encrypted_attributes

# Issue a credential for a user with encrypted attributes
def issue_credential(authority_private_key, encrypted_attributes):
    credential = {
        "encrypted_attributes": encrypted_attributes
    }
    credential_json = json.dumps(credential).encode('utf-8')
    signature = authority_private_key.sign(
        credential_json,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    credential['signature'] = base64.b64encode(signature).decode('utf-8')
    return credential

# Generate a token for accessing a web service using the credential
def generate_access_token(credential):
    expiration_time = datetime.now() + timedelta(days=1)  # Token expires in 1 day
    token_data = {
        "credential": credential,
        "expiration": expiration_time.isoformat()
    }
    token = base64.b64encode(json.dumps(token_data).encode('utf-8')).decode('utf-8')
    return token

# Generate RSA key pair for the authority
def generate_authority_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

# Generate a token for accessing a web service using the credential and permission
def generate_access_token(credential, permission):
    expiration_time = datetime.now() + timedelta(days=1)  # Token expires in 1 day
    token_data = {
        "credential": credential,
        "permission": permission,
        "expiration": expiration_time.isoformat()
    }
    token = base64.b64encode(json.dumps(token_data).encode('utf-8')).decode('utf-8')
    return token

def search_token_by_permission(tokens_by_permission, access_permission):
    if access_permission in tokens_by_permission:
        return tokens_by_permission[access_permission]
    else:
        return None  
              
def get_hosts_path():
    if os.name == 'nt':  # Windows
        return r"C:\Windows\System32\drivers\etc\hosts"
    else:  # Unix-based systems (Linux, macOS)
        return "/etc/hosts"
    
def block_websites(sites_to_block):
    hosts_path = get_hosts_path()
    redirect = "127.0.0.1"
    with open(hosts_path, 'r+') as hostfile:
        hosts_content = hostfile.read()
        for site in  sites_to_block:
            if site not in hosts_content:
                hostfile.write(redirect + ' ' + site + '\n')
    
def unblock_websites(sites_to_block):
    hosts_path = get_hosts_path()
    redirect = "127.0.0.1"
    with open(hosts_path, 'r+') as hostfile:
        lines = hostfile.readlines()
        hostfile.seek(0)
        for line in lines:
            if not any(site in line for site in sites_to_block):
                hostfile.write(line)
        hostfile.truncate()
    

def web_block(permission):
    hosts_dir = get_hosts_path()  # Assuming Windows for now
    redirect = "127.0.0.1"
    sites_to_block = ['www.facebook.com', 'facebook.com','www.youtube.com','youtube.com','www.loteriesdecatalunya.cat','loteriesdecatalunya.cat','www.bet365.es','bet365.es']
    sites_to_unblock=['www.facebook.com', 'facebook.com','www.youtube.com','youtube.com']
        
    if permission == "limited_access":
        print("Access Granted", "Welcome! You have access to limited content.")
        time.sleep(1)
        block_websites(sites_to_block)
        time.sleep(1)
        unblock_websites(sites_to_unblock) 
        
        
    elif permission == "child-friendly":
        print("Access Granted", "Welcome! You have access to child-friendly.")
        time.sleep(1)
        block_websites(sites_to_block)
            
    else:
        print("Access Granted", "Welcome! You have access to full_access.") 
        time.sleep(1)
        unblock_websites(sites_to_block)  
       
if __name__ == "__main__":
    data = [
        {"ID": "1", "birthdate": "2000-01-01", "ludopathy": "No"},
        {"ID": "2", "birthdate": "2008-05-05", "ludopathy": "No"},
        {"ID": "3", "birthdate": "1998-12-31", "ludopathy": "Yes"},
        {"ID": "4", "birthdate": "1994-10-30", "ludopathy": "Yes"},
        {"ID": "5", "birthdate": "2010-06-28", "ludopathy": "No"},
        {"ID": "6", "birthdate": "1998-12-31", "ludopathy": "No"},
        {"ID": "7", "birthdate": "200-01-01", "ludopathy": "1"}
    ]
    
    tokens_by_permission = {}

    authority_private_key, authority_public_key = generate_authority_key_pair()

    for person in data:
        eligibility_status = check_eligibility(person)
        if eligibility_status == "adult_without_ludopathy":
            # Construct user attributes
            user_attributes = [str(person['ID']), str(person['birthdate'])]

            # Encrypt user attributes with authority's public key
            encrypted_attributes = encrypt_attributes(user_attributes, authority_public_key)

            # Issue a credential for the user
            credential = issue_credential(authority_private_key, encrypted_attributes)

            # Generate an access token for the user's credential with full access permission
            access_token = generate_access_token(credential, "full_access")
            tokens_by_permission.setdefault("full_access", []).append(access_token)

        elif eligibility_status == "adult_with_ludopathy":
            # Construct user attributes
            user_attributes = [str(person['ID']), str(person['birthdate'])]

            # Encrypt user attributes with authority's public key
            encrypted_attributes = encrypt_attributes(user_attributes, authority_public_key)

            # Issue a credential for the user
            credential = issue_credential(authority_private_key, encrypted_attributes)

            # Generate an access token for the user's credential with limited access permission
            access_token = generate_access_token(credential, "limited_access")
            tokens_by_permission.setdefault("limited_access", []).append(access_token)

        elif eligibility_status == "child-friendly":
            # Construct user attributes
            user_attributes = [str(person['ID']), str(person['birthdate'])]

            # Encrypt user attributes with authority's public key
            encrypted_attributes = encrypt_attributes(user_attributes, authority_public_key)

            # Issue a credential for the user
            credential = issue_credential(authority_private_key, encrypted_attributes)

            # Generate an access token for the user's credential with child-friendly access permission
            access_token = generate_access_token(credential, "child-friendly_access")
            tokens_by_permission.setdefault("child-friendly_access", []).append(access_token)

        elif eligibility_status == "invalid_data":
            print("Person with ID:", person['ID'], "has invalid data. No access token issued.")
    
      
    with open("tokens_by_permission.txt", "w") as file:
        for permission, tokens in tokens_by_permission.items():
            file.write(f"Permission: {permission}\n")
            for token in tokens:
                file.write(f"Token: {token}\n")
            file.write("\n")
           
    user_token = input("Enter your access token: ")

    # Search for the token based on its access permission
    found_token = None
    for permission, tokens in tokens_by_permission.items():
        if user_token in tokens:
            found_token = user_token
            access_permission = permission
            break

    if found_token:
        print(f"Token found with permission '{access_permission}'")
        web_block(access_permission)
        webbrowser.open('https://www.google.com')
    else:
        print("Token not found or invalid.")
